<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries
This section is somewhat intimidating to new users, but modifications are necessary only to access advanced features.

In [1]:
!git clone --depth 1 https://github.com/profteachkids/chetools.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/chetools") #Path to CHE module imports

In [2]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
from jax.experimental.host_callback import id_print
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform2 import VSC, Comp, Range
import tools.che as che

## Dot-access to chemical properties and calculations
ChemCAD data properties and binary interaction parameters can be exported as text files.  A few common chemicals are stored in a Github repository.  Users with ChemCad access can export and access their own data files.  The *che.Props* class parses these files for convenient dot-access and implementations of common calculations.

In [3]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [4]:
print(f'Tc: {p.Tc}')
print(f'Pc: {p.Tc}')
print(f'Vapor Pressure at 432.1 K {p.Pvap(432.1)} Pa')
print(f'NRTL activity coefficients for equimolar mixture {p.NRTL_gamma([1/3,1/3,1/3], 300)} ')

Tc: [513.92 508.31 647.35]
Pc: [513.92 508.31 647.35]
Vapor Pressure at 432.1 K [1220670.06610508 1069634.05615236  600793.6517417 ] Pa
NRTL activity coefficients for equimolar mixture [1.02524645 1.13185085 2.06442906] 


## Adiabatic Flash Calculation
Model mass/energy balance and VLE - all variables (knowns and unknowns) are in a convenient DotMap structure (c).  The model is written once.

Sum up weighted square deviations for minimization.
 

In [5]:
def model(c):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    r=DotMap()
    r.V = c.Vy * c.Vtot # Moles of each component = mole fractions * total moles
    r.L = c.Lx * c.Ltot
    r.F = c.Fz * c.Ftot
    mass_balance = r.F - r.V - r.L # Mass balance for each component (vectors!)

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    r.FH = p.Hl(nL=r.F, T=c.FT)
    r.VH = p.Hv(nV=r.V, T=c.flashT)
    r.LH = p.Hl(nL=r.L, T=c.flashT)
    energy_balance = (r.FH - r.VH - r.LH)

    # Raoults with NRTL activity coefficient correction.  One-liner!
    r.fugL = c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT)
    r.fugV = c.Vy*c.flashP
    VLE = r.fugL - r.fugV
    id_print([mass_balance, energy_balance, VLE])
    return [mass_balance, energy_balance, VLE], r

## What's known (static) and unknown (variable) - automatically combined and tracked.

What's known and unknown can be easily swapped around without having to modify the model.  The DotMap structures can have nested lists of DotMaps with nested lists of arrays, and so forth.

In [6]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
c=DotMap()
c.Ftot=10 # Total Feed moles
c.Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
c.FT = 450 # Feed temperature
c.flashP= 101325 # Flash drum pressure

c.Vy = Comp(c.Fz) # Guess vapor/liquid composition equal to feed
c.Lx = Comp(c.Fz) # Comp - constrains mole fractions to behave like mole fractions!
c.flashT = Range(360, 273.15, c.FT)  # Guess and bounds for flash temperature
c.Vtot = Range(c.Ftot/2, 0., c.Ftot)  # Guess half of feed in vapor
c.Ltot = Range(c.Ftot/2, 0., c.Ftot)

## Magic happens here
The VSC class combines unknown variables and static parameters into a convenient DotMap structure.  Transformations between known/unknown DotMaps and the flat arrays required by the minimization routine is automated.

The minimization algorithm is a robust Trust-Newton Conjugate Gradient coded to take advantage of JAX automatic Jacobian vector products.  

The initial JAX model compilation is a bit slow, but subsequent repeat calculations are fast.

In [7]:
vsc=VSC(c, model)
vsc.solve(jit=True, verbosity=0)

[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [14479.44772707 11725.37795764  6218.03624225] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [14479.44772707 11725.37795764  6218.03624225] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [14479.44772707 11725.37795764  6218.03624225] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [14479.44772707 11725.37795764  6218.03624225] ]
[ [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
  -48834.08861275
  [14479.44772707 11725.37795764  6218.03624225] ]
[ [ 0.02868022  0.01402211 -0.04270234]
  -323.2227044
  [1456.68038042 1117.1461303   570.05309279] ]
[ [ 0.02868022  0.01402211 -0.04270234]
  -323.2227044
  [1456.68038042 1117.1461303   570.05309279] ]
[ [-3.92808400e-05  1.40196288e-04 -1.00915448e-04]
  -5.04784396
  [10.57574702 15.14659454 15.03627412] ]
[ [-3.92808400e-05  1.40196288e-04 -1.00915448e-04]
  -5.04784396
  [10.57574702 15.14659454 15.

In [8]:
# State parameters
vsc.sdf

Scalar   Vector3                    
                        1         1         2         3
Ftot                450.0                              
Fz                   10.0                              
FT                         0.333333  0.333333  0.333333
flashP  6.028361215767602

In [9]:
# Adjustable Variables
vsc.vdf

Scalar   Vector3                    
                         1         1         2         3
Vy                          0.321226  0.329191  0.349583
Lx       3.971638784232397                              
flashT                       0.35171  0.339621  0.308669
Vtot              101325.0                              
Ltot    352.85497498826226

In [10]:
# Intermediate results
vsc.rdf



Scalar       Vector3                            
                        1             1             2             3
V                              1.396864      1.348853      1.225922
L                              1.936469      1.984481      2.107411
F                              3.333333      3.333333      3.333333
FH     -2707524.043486322                                          
VH     -979268.7063295332                                          
LH    -1728255.3371567884                                          
fugL                       35636.992071  34412.118784  31275.889146
fugV                       35636.992071  34412.118784  31275.889146

In [11]:
vsc.cdf


Scalar   Vector3                    
                         1         1         2         3
FT                   450.0                              
Ftot                  10.0                              
Fz                          0.333333  0.333333  0.333333
Ltot     6.028361215767602                              
Lx                          0.321226  0.329191  0.349583
Vtot     3.971638784232397                              
Vy                           0.35171  0.339621  0.308669
flashP            101325.0                              
flashT  352.85497498826226

In [12]:
vsc.r.FH - vsc.r.VH - vsc.r.LH

DeviceArray(0., dtype=float64)

In [13]:
vsc.r.fugL - vsc.r.fugV

DeviceArray([3.63797881e-11, 2.11002771e-10, 4.58385330e-10], dtype=float64)